# Setup

This notebook illustrates how to access the SBOX-COST problems using IOHexperimenter.
To install IOHexperimenter, the following command can be used:

In [1]:
#%pip install ioh

Once installed, simply import the package

In [2]:
import ioh

# Accessing problems

In ioh, everything revolves around the problem class, so we start by creating an individual SBOX-COST problem

In [3]:
f = ioh.get_problem(1, instance=1, dimension=5, problem_class=ioh.ProblemClass.SBOX)

This problem contains all kinds of information about the way it is constructed. It also keeps track of all evaluations, which can be seen as follows:

In [4]:
f([0,0,0,0,0])
print(f.meta_data)
print(f.state)

<MetaData: Sphere id: 1 iid: 1 dim: 5>
<State evaluations: 1 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0] y: 99.51746200000001>>


For the SBOX-COST suite, the bounds are considered as hard constrains, so evaluating outside the domain will give an infinite value:

In [5]:
f([6,0,0,0,0])

inf

This is not the case for the BBOB version of the functions:

In [6]:
f_bbob = ioh.get_problem(1, instance=1, dimension=5, problem_class=ioh.ProblemClass.BBOB)
f_bbob([6,0,0,0,0])

125.27037568

Since this problem can be evaluated as a common python function, it should work directly with any optimizer. We make a basic random search to illustrate this:

In [7]:
import numpy as np

class RandomSearch:
    'Simple random search algorithm'
    def __init__(self, n: int, length: float = 0.0):
        self.n: int = n
        self.length: float = length
        
    def __call__(self, problem: ioh.problem.RealSingleObjective) -> None:
        'Evaluate the problem n times with a randomly generated solution'
        
        for _ in range(self.n):
            # We can use the problems bounds accessor to get information about the problem bounds
            x = np.random.uniform(problem.bounds.lb, problem.bounds.ub)
            self.length = np.linalg.norm(x)
            
            problem(x)   

If we want to run this algorithm on our problem, we can do the following:

In [8]:
r = RandomSearch(10)
r(f)
print(f.state)

<State evaluations: 12 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0] y: 99.51746200000001>>


To run multiple independent runs on the same problem, we can reset the state as follows:


In [9]:
f.reset()

# Logging

The default usage of IOHExperimenter is in generating logs of benchmarking experiments which can be analyzed in IOHAnalyzer.

In [10]:
import os

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="my-experiment",       # in a folder named: 'my-experiment'
    algorithm_name="random-search",    # meta-data for the algorithm used to generate these results
    store_positions=False               # disable storing x-variables in the logged files
)

# this automatically creates a folder 'my-experiment' in the current working directory
# if the folder already exists, it will given an additional number to make the name unique
logger

<Analyzer /home/jacob/code/IOHexperimenter/example/my-experiment>

We can add this logger to a problem so we can store the data when running our algorithm

In [11]:
f.attach_logger(logger)

In [12]:
r = RandomSearch(100)
r(f)

Once finished with the run, we can close the logger to force it to write the data (happens automatically when running as a python script)

In [13]:
logger.close()

# Experiment class

In Python, we provide the Experiment class which can be used to easily run a given algorithm over a larger number of problems.

In [14]:
experiment = ioh.Experiment(
    algorithm = RandomSearch(10), # An algorithm instance
    fids = [1, 2, 3],               # the id's of the problems we want to test
    iids = [1, 2],               # the instances 
    dims = [5, 20],               # the dimensions
    reps = 1,                     # the number of runs,
    problem_class=ioh.ProblemClass.SBOX, #the problem type
    zip_output = True       
)

Running this experiment creates a zip-file, which can directly be processed by IOHanalyzer (https://iohanalyzer.liacs.nl)

In [15]:
experiment()

For more background and other information on using IOHexperimenter, we refer to https://iohprofiler.github.io/IOHexp/

If you have any questions about the usage of SBOX-COST, or about IOHprofiler in general, please make an issue on https://github.com/sbox-cost/Examples

In [16]:
# cleanup
from shutil import rmtree
import os
import glob

def clean():
    for name in ("my-experiment", "ioh_data"):
        for path in glob.glob(f"{name}*"):
            if os.path.isfile(path):
                os.remove(path)
            if os.path.isdir(path):
                rmtree(path, ignore_errors=True)

def ls(p="./"):
    for obj in os.listdir(os.path.normpath(p)):
        print(obj)

def cat(f):
    with open(os.path.normpath(f)) as h:
        print(h.read())

clean()

rmtree("my-experiment", ignore_errors=True)
rmtree("ioh_data", ignore_errors=True)